In [2]:
import pandas as pd
import pandas as pd
import numpy as np
from numpy import mean
from numpy import std

from imblearn.over_sampling import SMOTE

from sklearn.model_selection import LeaveOneOut, cross_validate, cross_val_score, StratifiedShuffleSplit, train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.pipeline import Pipeline

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif

from pprint import pprint

import seaborn as sns
import matplotlib.pyplot as plt            # To visuvalizing the dat

import warnings
warnings.filterwarnings("ignore")

## Bioactive Peptides

In [3]:
df_train = pd.read_csv('/kaggle/input/predneuro-temporal-prob-dataset/train.csv')
df_test = pd.read_csv('/kaggle/input/predneuro-temporal-prob-dataset/test.csv')

In [4]:
X_train = df_train.drop(['label'], axis=1)
X_test = df_test.drop(['label'], axis=1)

y_train = df_train['label']
y_test = df_test['label']

In [5]:
y_train.value_counts()

label
1    1940
0    1940
Name: count, dtype: int64

In [6]:
y_test.value_counts()

label
0    495
1    495
Name: count, dtype: int64

In [8]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score, matthews_corrcoef

# Define Logistic Regression model and parameter grid
LR = LogisticRegression(random_state=42)
param_grid = {'penalty': ['l1', 'l2', 'elasticnet', 'none'],
              'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']}

# Perform Grid Search with cross-validation
lr_cv = GridSearchCV(LR, param_grid, scoring='accuracy', n_jobs=-1, verbose=1)
lr_cv.fit(X_train, y_train)

# Get the best estimator
lr = lr_cv.best_estimator_
print('Best accuracy and parameters are:', lr_cv.best_score_, lr_cv.best_params_)

# Predict on test set
print('\n Test Results \n')
y_p = lr.predict(X_test)

# Calculate AUC-ROC
y_prob = lr.predict_proba(X_test)[:, 1]  # Probability of positive class for AUC
auc_score = roc_auc_score(y_test, y_prob)
print("AUC-ROC Score:", auc_score)

# Calculate Matthews Correlation Coefficient (MCC)
mcc = matthews_corrcoef(y_test, y_p)
print("Matthews Correlation Coefficient (MCC):", mcc)

print(classification_report(y_test, y_p))

Fitting 5 folds for each of 20 candidates, totalling 100 fits


/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1173: FutureWarning: `penalty='none'`has been deprecated in 1.2 and will be removed in 1.4. To keep the past behaviour, set `penalty=None`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1173: FutureWarning: `penalty='none'`has been deprecated in 1.2 and will be removed in 1.4. To keep the past behaviour, set `penalty=None`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1173: FutureWarning: `penalty='none'`has been deprecated in 1.2 and will be removed in 1.4. To keep the past behaviour, set `penalty=None`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1173: FutureWarning: `penalty='none'`has been deprecated in 1.2 and will be removed in 1.4. To keep the past behaviour, set `penalty=None`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:11

Best accuracy and parameters are: 0.9649484536082475 {'penalty': 'l2', 'solver': 'liblinear'}

 Test Results 

AUC-ROC Score: 0.9554861748801142
Matthews Correlation Coefficient (MCC): 0.7900602266816243
              precision    recall  f1-score   support

           0       0.90      0.88      0.89       495
           1       0.89      0.91      0.90       495

    accuracy                           0.89       990
   macro avg       0.90      0.89      0.89       990
weighted avg       0.90      0.89      0.89       990



/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


In [9]:
from sklearn.metrics import confusion_matrix

# Assuming y_test and y_pred are already defined
tn, fp, fn, tp = confusion_matrix(y_test, y_p).ravel()

# Sensitivity (Recall or True Positive Rate)
sensitivity = tp / (tp + fn)

# Specificity (True Negative Rate)
specificity = tn / (tn + fp)

print(f"Sensitivity: {sensitivity:.4f}")
print(f"Specificity: {specificity:.4f}")

Sensitivity: 0.9051
Specificity: 0.8848
